Exercise (part 1) – implementation of FBA problem

Consider the iML1515 stoichiometric model of Escherichia coli K-12 substr.
MG1655 in SBML format (http://bigg.ucsd.edu/models/iML1515).
The compound of interest for your group and oxygen conditions are available
in the table below.
Group 4: Formate(mmol/gDW/h), Oxygen(20 mmol/gDW/h), Glucose(15 mmol/gDW/h)

1. What is the wild-type production of your group’s compound?

In [ ]:
from mewpy.simulation import get_simulator
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml')
print(model.summary())
envcond = {'EX_o2_e': (-20.0, 100000.0), 'EX_glc__D_e': (-15.0,100000.0)}
simul= get_simulator(model, envcond=envcond)
result= simul.simulate(method='FBA')
print(result.fluxes['EX_for_e'])

Efetuando um Flux Balance Analysis (FBA) com as condições ambientais dadas, constatamos que a E.coli wild-type não produz formato.

2. What are the maximum compound production capabilities?

In [ ]:
print(simul.FVA(reactions=['EX_for_e'], format='df'))

Efetuando um Flux Variability Analysis (FVA) com as condições ambientais dadas, observamos que este modelo consegue produzir um máximo de 28,29(fluxo) de formato.

3. Use different optimization objective functions to improve the production
   of the compound, considering that cells have evolved for maximum
   growth.

    a.Evaluate if single gene deletions enhance the production of the
    compound. Rank the mutants obtained according to the
    compound production capacity and growth performance.

In [ ]:
PRODUCT_ID = 'EX_for_e'
BIOMASS_ID = 'BIOMASS_Ec_iML1515_core_75p37M'
from mewpy.optimization.evaluation import BPCY, TargetFlux
evaluator_1 = BPCY(BIOMASS_ID, PRODUCT_ID, method='pFBA')
evaluator_2 = TargetFlux(PRODUCT_ID)
from mewpy.problems import GKOProblem #gene knockout
problem = GKOProblem(model, fevaluation=[evaluator_1, evaluator_2], envcond=envcond, candidate_max_size=1)
from mewpy.optimization import EA
mutante = EA(problem, max_generations=50, visualizer=True)
mutante_run= mutante.run()
print(mutante_run)
from mewpy.utils.constants import ModelConstants
ModelConstants.RESET_SOLVER = True

Após efetuar uma simulação com o objetivo de maximizar a produção de formato e executando a deleção de um gene, observou-se que a produção de formato não aumenta.
Segundo a simulação, o nocaute do gene b3733 resulta num fluxo de 11,42, utilizando a função de avaliação BCPY (Biomass-Product Coupled Yield). Este é inferior ao fluxo extrapolado na FVA, o que indica que realizar a deleção de apenas um gene, não trará vantagens à produção de formato.

    b. Determine the best strategy, up to five modifications, to improve
       the compound production.

In [ ]:
PRODUCT_ID = 'EX_for_e'
BIOMASS_ID = 'BIOMASS_Ec_iML1515_core_75p37M'
from mewpy.optimization.evaluation import BPCY, TargetFlux
evaluator_1 = BPCY(BIOMASS_ID, PRODUCT_ID, method='pFBA')
evaluator_2 = TargetFlux(PRODUCT_ID)
from mewpy.problems import RKOProblem
problem = RKOProblem(model, fevaluation=[evaluator_1, evaluator_2], envcond=envcond, candidate_max_size=5)
from mewpy.optimization import EA
mutante2 = EA(problem, max_generations=50, visualizer=True)
mutante_run2= mutante2.run()
print(mutante_run2)
from mewpy.utils.constants import ModelConstants
ModelConstants.RESET_SOLVER = True

A melhor estratégia para aumentar a produção de formato seria fazer um nocaute a 4 reações (H2Otex, SGDS, CRNBTCT e PLIPA2A180pp), que resultaria num fluxo de 49.36, valor este superior ao obtido no FVA wild-type.

4. Analyse the strategies proposed by your optimization, from the
   metabolic viewpoint. Would it be feasible to implement this strategy in
   the lab? Why?

De acordo com as simulações efetuadas, a estratégia que resultava numa maior produção de formato seria efetuar o nocaute de 4 reações.
Esta estratégia não parece viável para se executar num laboratório devido à elevada complexidade envolvida neste processo.
